In [1]:
import os
import chromadb

from together import Together
from openai import OpenAI

import rag_functions as rf

In [2]:
models_path = "models/models_test.json"
documents_path = "dataset/texts"
qa_eval_set_path = "dataset/qa/qa_eval_set.json"

In [3]:
embedding_models = rf.list_models(models_path, models_type="embedding_models")
generative_models = rf.list_models(models_path, models_type="generative_models")

In [4]:
qa_eval_set = rf.load_json(qa_eval_set_path)

In [5]:
texts = rf.load_texts(documents_path)


In [6]:
chunk_data = rf.chunk_texts(texts, chunk_size=128, overlap=10)
chunk_names, text_chunks = rf.dict_to_kv_lists(chunk_data) 

Class for saving vectors of documents using a specified model.

In [ ]:
class Vectorizator:
    def __init__(self, documents, ids):
        self.documents = documents
        self.ids = ids
        self.chroma_client = chromadb.PersistentClient(
            path="chroma_data/"
        )
        
    def get_or_load_model_collection(self, model_family, embedding_model_name, save=False):
        collection_name = f"collection_by_{embedding_model_name}"
        if save:
            try:
                self.populate_db(collection_name, model_family, embedding_model_name)
            except Exception as err:
                if "exists" in str(err):
                    print(f"Collection already exist – loading it instead.")
                    self.load_collection(collection_name)
                else:
                    print(f"An unknown error occurred while saving:\n{err}")
                    raise  
        else:
            try:
                self.load_collection(collection_name)
            except Exception as err:
                print(f"Error:\n{err}\nDatabase will be populated instead.")
                self.populate_db(collection_name, model_family, embedding_model_name)

    def populate_db(self, collection_name, model_family, embedding_model_name):
        ef = rf.universal_ef(model_family, embedding_model_name)
        self.collection = self.chroma_client.create_collection(
            name=collection_name, 
            embedding_function=ef()
        )
        self.collection.upsert(
            documents=self.documents,
            ids=self.ids
        )
        print(f"Documents successfully embedded and saved to a collection.")
        
    def load_collection(self, collection_name):
        self.collection = self.chroma_client.get_collection(collection_name)
        print(f"Documents successfully loaded as a collection.")
            
    def get_results(self, question, n_results=3):
        outputs = self.collection.query(
            query_texts=[question], 
            n_results=n_results
        )
        return outputs

In [23]:
vectorizator = Vectorizator(
    documents=text_chunks,
    ids=chunk_names
)

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
vectorizator.get_or_load_model_collection("openai_embedding", "text-embedding-3-large", save=False)

Documents successfully loaded as a collection.


In [28]:
vectorizator.get_results("Co ten Neptun?")

InvalidArgumentError: Collection expecting embedding with dimension of 3072, got 384

In [ ]:
for model_family in embedding_models:
    for model_name in model_family:
        vectorizator.get_or_load_model_collection(model_family, model_name, save=False)